In [10]:
%%time
p = 2^521 - 1
# proof.arithmetic(False)
F = GF(p)
A = p-3
B = 1093849038073734274511112390766805569936207598951683748994586394495953116150735016013708737573759623248592132296706313309438452531591012912142327488478985984
# q = 6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
q = E.cardinality()
# Nn = 6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
E = EllipticCurve([F(A), F(B)])

CPU times: user 4min 27s, sys: 157 ms, total: 4min 28s
Wall time: 4min 29s


In [11]:
def stringPoint_encode(E, string):
    A = E.a4()
    B = E.a6()
    p = E.base_field().cardinality()
    b = 2^8
    msg = str(string)
    n = len(msg)
    d = 100
    pts = []
    for t in range(n // 64 + 1):
        m_current = msg[64 * t : 64 * (t + 1)]
        m = sum(ord(m_current[k]) * b^k for k in range(min(len(m_current), 64)))
        for j in range(d):
            x = (d * m + j) % p
            s = (x^3 + A * x + B) % p
            if s == power_mod(s, int((p+1)/2), p):
                y = power_mod(s, int((p+1)/4), p)
                pts.append(E([x,y]))
                break
    return pts
def pointString_decode(lst_P):
    b = 2^8
    d = 100
    lst = []
    for P in lst_P:
        m = int((floor((P[0]/P[2])))/d)
        while m != 0:
            lst.append(chr(m % b))
            m //= b
            m = int(m)
    return ''.join(lst)

In [12]:
m = 'Oi Bob, tudo bom? Como vai voce? O numero maximo e 64 caracteresa'
pointString_decode(stringPoint_encode(E, m))

'Oi Bob, tudo bom? Como vai voce? O numero maximo e 64 caracteresa'

In [13]:
pointString_decode(stringPoint_encode(E, 'criptografia uhul!'))

'criptografia uhul!'

In [14]:
def base_point(E):
    P = E.random_point()
    #q = E.cardinality()
    while P.order() != q:
        P = E.random_point()
        print(P)
    return P

In [17]:
# %%time
P = base_point(E)
print(u'O ponto base é: ')
print(P)

O ponto base é: 
(5001049121673435275507178074207699748297783798871320432107191255306302642485280079037913280024336222384127933942415715658283282719952954885132378427891236046 : 6607598958016334332621839861277005115877074579282994063890773142238419919572868058521874135977555733633335818141049003169761511001884602207026569632200378435 : 1)


In [20]:
def generate_keys(E, P):
#     Nn = E.cardinality()
#     print(Nn)
    a = ZZ.random_element(q)
    while gcd(a, q) != 1:
#         print(a)
        a = ZZ.random_element(q)
    return [a, a * P]

In [21]:
# %%time
alice_keys = generate_keys(E, P)

print(u'Chave Privada (Alice): %s \n \n' %(alice_keys[0]))
print(u'Chave Pública (Alice):')
print(alice_keys[1])
print('\n')



Chave Privada (Alice): 3975803846256631968271129881026223949928236871926809879553431922303290700097095094054199339932402877108126359532713745836734374719641556380290968590207786248 
 

Chave Pública (Alice):
(6113129035879123844227030618129313744058067245401155916133508346088138789806992650822420380253175207610182388446277511486731351420951829795216708915289422492 : 388424682695211051390709610685335120961593269587928201549407734671919541830077762716332976060408702951640406771550320032767966282932631836542535049972239245 : 1)




In [22]:
# %%time
bob_keys = generate_keys(E, P)

print(u'Chave Privada (Bob): %s \n \n' %(bob_keys[0]))
print(u'Chave Pública (Bob):')
print(bob_keys[1])
print('\n')

Chave Privada (Bob): 4433996417410345515706570732994485816646975768601860419810793121566630208191122785852830738983140283518607758127929352248053846642145655407879600169896181817 
 

Chave Pública (Bob):
(1199586487486048229287548522341999580972865119725302795592029801056031026423901627374743410571999225301717787450173613663298776175366090869650074021973350201 : 1937182586883752810136706964884893173580603803424536570893475604781622322951373348911733095492676320157048361249769081957323235026904925452249690656548326617 : 1)




In [27]:
def encode(E, P, msg, privateKey_sender, publicKey_sender, publicKey_reciever):
    PM = stringPoint_encode(E, msg)
    enc_msg = []
    for Q in PM:
        k = ZZ.random_element(q)
        kP = k *  publicKey_sender
        enc_msg.append([kP, Q + k * privateKey_sender * publicKey_reciever])
#     enc_msg = [kP, PM + k * privateKey_sender * publicKey_reciever]
    return enc_msg

def decode(E, P, enc_msg, publicKey_sender, privateKey_reciever):
    lst_P = []
    for e_msg in enc_msg:
        kP = privateKey_reciever * e_msg[0]
        PM = e_msg[1] - kP
        lst_P.append(PM)
    dec_msg = pointString_decode(lst_P)
    return dec_msg
    
    

In [30]:
t0 = encode(E, P, 'oi bob tudo bom?', alice_keys[0], alice_keys[1],bob_keys[1])[0][0]
t1 = encode(E, P, 'oi bob tudo bom?', alice_keys[0], alice_keys[1],bob_keys[1])[0][1]

print('Mensagem encriptada, primeira parte: ')
print(t0)
print('\n Mensagem encriptada, segunda parte:')
print(t1)

Mensagem encriptada, primeira parte: 
(2818798398535372688312153504552693810474964012391455277493213488597508264268246615101357478231165184174403251923878702885361714185973346396580955024090662131 : 1253341389999043705554429565923332433091379184059964454915488340981362831885362196495639076971157506836759467261812822719241644987896635213289139862302070581 : 1)

 Mensagem encriptada, segunda parte:
(6340305756934536020027640581090795304561387788333995440709102000372807102965187042773433058522133223144920839295631263941111445304245075558211506324387383324 : 5086052101325723575594254609752197705741094193238241464642588359288075524512943216860802957024577878243137007941879311416130588727732072957223076774602691318 : 1)


In [35]:
decode(E, P, encode(E, P, 'Oi Bob, tudo bom? Como vai voce? O numero maximo e 64 caracteres', alice_keys[0], alice_keys[1], bob_keys[1]), alice_keys[1], bob_keys[0])

'Oi Bob, tudo bom? Como vai voce? O numero maximo e 64 caracteres'

In [46]:
%%time
pts = encode(E, P, '...', alice_keys[0], alice_keys[1], bob_keys[1])

CPU times: user 1min 51s, sys: 6.94 s, total: 1min 58s
Wall time: 1min 58s


2

In [51]:
pointString_decode([p[1] for p in pts])

'.K\x9f\xec\xc9\x94\x1f\xe5\x17\x02\xb5\xbb\xc3i\xdd;\xcc\xbd_\x01Q\x8f\xc7J\x90}=\x02\x9b\x8bS\'\x93\xc2\x9b\xa8\xe8\xde\xcc\x91&A\x8f\xd6\xfb\x8e\x08\\\xcaR\xb3\xcb\xbd\x9b!\xea\xb29\x05\xbc\x93Ue\xc8NY\xac\xdby\xc6\x10\xe6<\xa7)\x10\xee \xde_\\Q\xe6\xb9X\xfc\x18\x8e\xa2z\xc2\xbf\xcf\xb6\xbb\xb8\xb9\xcd\xe6\xa2\xdc\xcd`&R>\xd5T\x8f\x12\x0f\xa8\xba/~.4\xc7:\xc9LI\xf9}\xa5dR\x01\x01\xec\xffs\xbd\x1c\x0b]\x85\x88\x87\xfc\x97\xed&\xc6\x86;jdl\xe7\xd0\x06\xf4\xc5\x9b\xd1S,\x0cat\x05.\x00\xe1\x8b5\x80?\xb0\x98"K\xe9\xb6\x04y\xcc\xd2J\x01&0\xd9\x95g\xe1\xc3\xff\t[\xca\xfc\x01d\xe8\x9f)\xbfd\x18\x98\xc0r\x85\xfes\xcb\x98\nsY\xd9\xd4m\x8d\x99!\xa0\xebU\xa9\xdbZ\x97gD2\xd8\x91\xa3\xac4\xb2\xc4\xc2\xb0\x8b\xe8\xc1\x17\xc7\xe5\xc8|\x07\xa7\xc2\xdd\n&\xd5\xb8m*\xc7\xd1\xfb\xda\x9a\x138\xef\xb0G\xcf1\x82\xacL{&W\xa7\xa4\xe6C\x8f\x9dR\xc4\x1e\xbd\xb9U_\xbe \xe5k\xc9\xdck\xcf\xff\x87f\xa1\xbb\xc9D\x1fH\xa9\x83\xe0\x19\xfeQ;\x89 \x83\xe6\x8cp\xde}\x1d\x1c\xf0!\x03\x15\\x\x86y\x1c\xd0#\xff\x13\xdf\x9f

In [36]:
from PIL import Image

In [42]:
Image.open(r'imgs/alice.png')

IOError: decoder jpeg not available

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=820x1548 at 0x6FBEB9B8B50>